# Play with keycloak

We will use `python-keycloak-client` to play with keycloak. Make sure you have created a **REALM** and **Client** in Keycloak.
If you don't know how, please check this [doc](../docs/Install_keycloak.md).


## Step1. Create a keycloak oidc client

To create a keycloak oidc client, you need to know
- url of the keycloak server
- the name of the realm
- client id
- client secret key

In [1]:
from keycloak import KeycloakOpenID

In [2]:
url = "http://localhost:8080"
my_realm_name = "pengfei-test"
client_id = "pengfei-dv-app"
client_secret = "8BJOcHtXlXpKVLGFOjGoAJ4nxo1WyNKT"

In [3]:
oidc_client = KeycloakOpenID(server_url=url, realm_name=my_realm_name, client_id=client_id,
                             client_secret_key=client_secret)

## Step2. Get a token for a given user

In [4]:
user_name = "pengfei"
user_secret = "toto"

In [5]:
token = oidc_client.token(user_name, user_secret)
print(token)

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJvYjE2ZW9kOEZZeFVqOFdUaXRid295c09PSC11MDF1VVBwZGdPUVBWLXJzIn0.eyJleHAiOjE2NTI2OTM5ODcsImlhdCI6MTY1MjY5MzM4NywianRpIjoiMTlmYWQxNDktZjg1Zi00M2RjLTg4YzktOGI5ODlkYjRhOTAzIiwiaXNzIjoiaHR0cDovL2xvY2FsaG9zdDo4MDgwL3JlYWxtcy9wZW5nZmVpLXRlc3QiLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiNWE1ZjYxMTktMDY5YS00NDk3LWFkMWItMWZjMDNiNDVjMjcyIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoicGVuZ2ZlaS1kdi1hcHAiLCJzZXNzaW9uX3N0YXRlIjoiM2U2MTFhYjYtYTBkMC00MjBjLWFmMGYtN2ZjOGY2M2VjZjhlIiwiYWNyIjoiMSIsImFsbG93ZWQtb3JpZ2lucyI6WyJodHRwczovL2xvY2FsaG9zdDo4ODg4Il0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1wZW5nZmVpLXRlc3QiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6ImVtYWlsIHByb2ZpbGUiLCJzaWQiOiIzZTYxMWFiNi1hMGQwLTQyMGMtYWYwZi03ZmM4ZjYzZWNmOGUiLCJlbWFpbF92ZXJpZmllZCI6ZmFsc2UsIm5hbWUiOiJwZW5nZmVpIGxpdSIsInByZWZlcnJlZF

## Step3. Get access token

You can notice that the token has many infomation, and it allows you to renew your access token. The user information is located in the access token. We can get the token.

Let's first use the built-in parser to view the access token

In [6]:
access_token = token['access_token']

In [7]:
userinfo = oidc_client.userinfo(access_token)

In [8]:
print(userinfo)

{'sub': '5a5f6119-069a-4497-ad1b-1fc03b45c272', 'email_verified': False, 'name': 'pengfei liu', 'preferred_username': 'pengfei', 'given_name': 'pengfei', 'family_name': 'liu'}


You can notice we don't have full information about the user. Let's parse the token with our own parser

In [9]:
from src.JWTTokenValidator import JWTTokenValidator

In [10]:
token_validator=JWTTokenValidator()
user_info=token_validator.get_token_payload_without_verification(access_token)

In [11]:
print(user_info)

{'exp': 1652693987, 'iat': 1652693387, 'jti': '19fad149-f85f-43dc-88c9-8b989db4a903', 'iss': 'http://localhost:8080/realms/pengfei-test', 'aud': 'account', 'sub': '5a5f6119-069a-4497-ad1b-1fc03b45c272', 'typ': 'Bearer', 'azp': 'pengfei-dv-app', 'session_state': '3e611ab6-a0d0-420c-af0f-7fc8f63ecf8e', 'acr': '1', 'allowed-origins': ['https://localhost:8888'], 'realm_access': {'roles': ['offline_access', 'uma_authorization', 'default-roles-pengfei-test']}, 'resource_access': {'account': {'roles': ['manage-account', 'manage-account-links', 'view-profile']}}, 'scope': 'email profile', 'sid': '3e611ab6-a0d0-420c-af0f-7fc8f63ecf8e', 'email_verified': False, 'name': 'pengfei liu', 'preferred_username': 'pengfei', 'given_name': 'pengfei', 'family_name': 'liu'}


In [12]:
token_validator.show_token_exp_time(access_token)

Token expiration time in unix ts: 1652693987
Token expiration time in UTC format: 2022-05-16 09:39:47
